# Analyzing credit card complaints data

Following blog
https://www.dataquest.io/blog/sql-intermediate

In [1]:
import pandas as pd

# psycopg2 lets us easily run commands against our db

import psycopg2
conn = psycopg2.connect("dbname=consumer_complaints user=oracle")
conn.autocommit = True
cur = conn.cursor()

In [2]:
def run_command(command):
    cur.execute(command)
    return cur.statusmessage

In [3]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://oracle@localhost/consumer_complaints')


In [4]:
def run_query(query):
    return pd.read_sql_query(query,con=engine)

## CREDIT CARD Complaints

In [7]:
query = 'SELECT * FROM credit_card_complaints LIMIT 5;'
run_query(query)

,complaint_id,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed
0,469026,2013-07-29,Credit card,None,Billing statement,None,None,None,Citibank,OH,45247,None,N/A,Referral,2013-07-30,Closed with explanation,Yes,Yes
1,469131,2013-07-29,Credit card,None,APR or interest rate,None,None,None,Synchrony Financial,WA,98548,None,N/A,Web,2013-07-29,Closed with monetary relief,Yes,No
2,479990,2013-07-29,Credit card,None,Delinquent account,None,None,None,Amex,TX,78232,None,N/A,Web,2013-07-29,Closed with monetary relief,Yes,No
3,475777,2013-07-29,Credit card,None,Billing disputes,None,None,None,Capital One,FL,32226,Servicemember,N/A,Web,2013-07-29,Closed with explanation,Yes,No
4,469473,2013-07-29,Credit card,None,Credit line increase/decrease,None,None,None,Citibank,WI,53066,Older American,N/A,Phone,2013-07-30,Closed with explanation,Yes,Yes


## Bank Account Complaints

In [8]:
query = 'SELECT * FROM bank_account_complaints LIMIT 5;'
run_query(query)

,complaint_id,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed
0,468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,None,None,None,Wells Fargo & Company,CA,95992,Older American,N/A,Web,2013-07-31,Closed with explanation,Yes,No
1,468879,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",None,None,None,Santander Bank US,NY,10065,None,N/A,Fax,2013-07-31,Closed,Yes,No
2,468949,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,None,None,None,Wells Fargo & Company,GA,30084,None,N/A,Web,2013-07-30,Closed with explanation,Yes,No
3,468981,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,None,None,None,Bank of America,TX,75025,None,N/A,Web,2013-07-30,Closed with explanation,Yes,No
4,469185,2013-07-29,Bank account or service,Other bank product/service,Deposits and withdrawals,None,None,None,Wells Fargo & Company,CA,96088,None,N/A,Web,2013-08-01,Closed with explanation,Yes,No


How many records in each table have null values for the consumer complaint narrative field

In [14]:
# credit card complaints
query = 'SELECT count(*) FROM credit_card_complaints WHERE consumer_complaint_narrative IS NOT NULL;'
run_query(query)

,count
0,17433


In [18]:
query ='SELECT count(*) FROM credit_card_complaints WHERE consumer_complaint_narrative IS NULL;'
run_query(query)

,count
0,70285


In [19]:
# bank account complaints
query = 'SELECT count(*) FROM bank_account_complaints WHERE consumer_complaint_narrative IS NOT NULL;'
run_query(query)

,count
0,13860


**VIEW**


view is essentially a logical representation of a query’s result. It behaves like a traditional table where you can select values, but you cannot insert into, update, or delete from it. Here are a few benefits of using views:

**Reusability** – Views can be used to reuse complex queries that are frequently used.

**Security** – Views are often made accessible to certain users so that they cannot view the underlying tables and only relevant data is available.

**Query Performance** – Sometimes, writing complex queries consisting of several subqueries and aggregations will utilize a large amount of resources.

In [21]:
command = 'CREATE VIEW credit_card_w_complaints AS SELECT * FROM credit_card_complaints WHERE consumer_complaint_narrative IS NOT NULL;'
run_command(command)

'CREATE VIEW'

In [22]:
command = '''
CREATE VIEW credit_card_wo_complaints as
    SELECT * FROM credit_card_complaints
    WHERE consumer_complaint_narrative IS NULL;
'''
run_command(command)

'CREATE VIEW'

In [23]:
command = '''
CREATE VIEW bank_account_w_complaints AS
    SELECT * FROM bank_account_complaints
    WHERE consumer_complaint_narrative IS NOT
 NULL;
'''
run_command(command)

'CREATE VIEW'

In [24]:
command = '''
CREATE VIEW bank_account_wo_complaints AS
    SELECT * FROM bank_account_complaints
    WHERE consumer_complaint_narrative IS
 NULL;
'''
run_command(command)

'CREATE VIEW'

In [25]:
query = 'SELECT * FROM bank_account_w_complaints LIMIT 5;'
run_query(query)

,complaint_id,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed
0,1297629,2015-03-24,Bank account or service,Checking account,Using a debit or ATM card,None,I 'm no longer an account holder of SunTrust. ...,Company chooses not to provide a public response,"SunTrust Banks, Inc.",FL,334XX,None,Consent provided,Web,2015-03-24,Closed with explanation,Yes,No
1,1297676,2015-03-24,Bank account or service,Checking account,"Account opening, closing, or management",None,"After banking with Citibank for over 3 years, ...",None,Citibank,TX,797XX,None,Consent provided,Web,2015-03-24,Closed with explanation,Yes,No
2,1297773,2015-03-24,Bank account or service,Other bank product/service,Deposits and withdrawals,None,On XXXX XXXX I went to the Citizens Bank on XX...,None,"Citizens Financial Group, Inc.",MA,021XX,None,Consent provided,Web,2015-03-24,Closed with explanation,Yes,No
3,1297972,2015-03-24,Bank account or service,Other bank product/service,"Account opening, closing, or management",None,The following summarizes my issue with PNC Ban...,None,PNC Bank N.A.,MI,493XX,Older American,Consent provided,Web,2015-03-30,Closed with explanation,Yes,No
4,1298081,2015-03-24,Bank account or service,Other bank product/service,"Account opening, closing, or management",None,This is a concern with companies in general. T...,None,Capital One,CA,906XX,None,Consent provided,Web,2015-03-24,Closed with explanation,Yes,No
